In [2]:
import hashlib
import random
import time


def ecdsa_sign(msg):
    """
    To sign a message, we need to hash the message with SHA-256
    Then to compute R, we take k * G
    
    Now, we need to compute the signature r and s
    To have r, we take Rx --> R[0] modulo N
    To have s, we take modulo inverse of k & N * (hash(message)+(r*private_key) % N)
    """
    e = int(hashlib.sha256(msg.encode()).hexdigest(),16)
    
    R = k * G
    
    r = int(R[0]) % N
    
    s = modinv(k, N) * (e+(r*d)) % N
    
    if r == 0 or s == 0:
        return False
    
    return r, s
    
def ecdsa_verify(msg, r, s):
    """
    To verify the signature of a message, we need to retrieve r from the public_key
    To do that, we hash the message
    And we do r_prime = (hash(message)* modulo inverse s & N) * G + (r * modulo inverse s & N) * public_key
    If r_prime == r --> The signature is verified
    """
    e = int(hashlib.sha256(msg.encode()).hexdigest(),16)
    
    r_prime = (e * modinv(s, N)) * G + (r * modinv(s, N)) * public_key
    print(f"r_prime = {r_prime[0]}")
    
    return r_prime[0] == r
    

def attack(list1, list2):
    """
    To attack ECDSA, we need to have a repeated nonce k
    Here we reuse k in two different message and we hash and sign them
    
    To retrieve the private key, we compute a value such that
    value = modulo inverse r1 * (s1 - s2) % N
    
    Then we retrieve the private key with:
    private_key = ((s2 * hash(message1)-s1*hash(message2))* value) % N 
    """
    e1 = int(hashlib.sha256(list1[0].encode()).hexdigest(),16)
    e2 = int(hashlib.sha256(list2[0].encode()).hexdigest(),16)
    
    value = modinv(list1[1] * (list1[2]-list2[2]), N)
    
    recover_private_key = ((list2[2]*e1-list1[2]*e2) * value) % N
    
    return recover_private_key
    

def modinv(a, b):
    """
    Here we compute the modolar inverse of a & b
    such that x = a**-1 % b
    """
    L1 = [a, 1, 0]
    L2 = [b, 0, 1]

    while L2[0] > 0:
        L = L2[:]
        q = L1[0] // L2[0]

        for i in range(0, 3):
            L2[i] = L1[i] - q * L2[i]
        L1 = L

    return L1[1]


if __name__ == '__main__':
    print(
"""
###########################################
# ECDSA Implementation  &                 #
# Repeated Nonce Attack                   #
# By Edouard Bettignies Masi-M1           #
###########################################
"""
    )
    
    P = 2 ** 256 - 2 ** 224 + 2 ** 192 + 2 ** 96 - 1
    A = P - 3
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    N = 115792089210356248762697446949407573529996955224135760342422259061068512044369
    Gx = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    Gy = 36134250956749795798585127919587881956611106672985015071877198253568414405109

    ZZ = Integers()
    Fp = GF(P)
    Fn = GF(N)

    NISTP256 = EllipticCurve(Fp, [A, B])
    NISTP256.set_order(115792089210356248762697446949407573529996955224135760342422259061068512044369)
    G = NISTP256(Gx, Gy)

    """Begin new code"""
    
    d = random.randint(1, N-1) # private key
    k = random.randint(1, N-1) # Nonce
    
    public_key = G * d
    
    list1 = []
    list2 = []
    
    choose_mode = input("Choose your mode sign & verify / attack (s/a) : ")
    
    start_time = time.time()
    
    if choose_mode == "s":
        f = open("plaintext.txt", "r")
        msg = f.read()
        
        try:
            r, s = ecdsa_sign(msg)
        except:
            print("r ou s = 0 !")
            
    elif choose_mode == "a":
        f1 = open("plaintext.txt", "r")
        msg_1 = f1.read()
        
        f2 = open("plaintext2.txt", "r")
        msg_2 = f2.read()
        
        try:
            r1, s1 = ecdsa_sign(msg_1)
            r2, s2 = ecdsa_sign(msg_2)
            
            list1 = [msg_1, r1, s1]
            list2 = [msg_2, r2, s2]
            
        except:
            print("r ou s = 0 !")
        
    
    print(f"\nPublic Key = {public_key}")
    print(f"Private Key = {d}\n")
    
    if choose_mode == "s":
        print(f"r = {r}")
        print(f"s = {s}")
        if ecdsa_verify(msg, r, s):
            print(f"\nSignature matches in {time.time() - start_time} seconds !")
        else:
            print("\nSignature failed !")
            
    elif choose_mode == "a":
        recover_private_key = attack(list1, list2)
        
        print(f"r1 = {r1}")
        print(f"s1 = {s1}\n")
        print(f"r2 = {r2}")
        print(f"s2 = {s2}")
        print(f"Recover private key = {recover_private_key}")
        
        if recover_private_key == d:
            print("\nAttack finished and worked !!!")
        else:
            print("\nAttack finish but didn't work")
    
    


###########################################
# ECDSA Implementation  &                 #
# Repeated Nonce Attack                   #
# By Edouard Bettignies Masi-M1           #
###########################################

Choose your mode sign & verify / attack (s/a) : a

Public Key = (20715357604667194527908150476935101552723764587817398932496928145969974316923 : 108617994300368185844285837037678899645407445696780542535778711979554870884894 : 1)
Private Key = 25022180185435856745660843376004390080425034409869348527444881555050211864681

r1 = 9994469001927958213774648180834542549998868661585760091608787448166584544863
s1 = 20485961276420989849598133525010038703010183677349442046160045002184495639884

r2 = 9994469001927958213774648180834542549998868661585760091608787448166584544863
s2 = 46958935379736664956158446018823475978646028615597353511300804756639846892137
Recover private key = 25022180185435856745660843376004390080425034409869348527444881555050211864681

Attack finished and work